In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
with open('data/get_movies.pkl', 'rb') as f:
    t = pickle.load(f)
    
movie_dic = t[0]
movie_genre_dic = t[1]

with open('data/get_ratings.pkl', 'rb') as f:
    lst = pickle.load(f)

In [3]:
def grad_U(Ui, Yij, Vj, y_mean, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return (1-2*reg*eta)*Ui + 2 * eta * Vj * (Yij - y_mean - np.dot(Ui,Vj))     

def grad_V(Vj, Yij, Ui, y_mean, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return (1-2*reg*eta)*Vj + 2 * eta * Ui * (Yij - y_mean - np.dot(Ui,Vj))

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    
    y_mean = np.mean(np.array([tup[2] for tup in Y]))
    
    # Compute mean squared error on each data point in Y; include
    # regularization penalty in error calculations.
    # We first compute the total squared squared error
    err = 0.0
    for (i,j,Yij) in Y:
        err += (Yij - y_mean - np.dot(U[i-1], V[:,j-1]))**2
    # Add error penalty due to regularization if regularization
    # parameter is nonzero
    if reg != 0:
        U_frobenius_norm = np.linalg.norm(U, ord='fro')
        V_frobenius_norm = np.linalg.norm(V, ord='fro')
        err += reg * (U_frobenius_norm ** 2)
        err += reg * (V_frobenius_norm ** 2)
    # Return the mean of the regularized error
    return err / float(len(Y))

def get_msqe_err(U, V, Y):
    y_mean = np.mean(np.array([tup[2] for tup in Y]))
    
    err = 0.0
    for (i,j,Yij) in Y:
        err += (Yij - y_mean - np.dot(U[i-1], V[:,j-1]))**2
    
    err /= len(Y)
    
    return err ** 0.5

def train_model(M, N, K, eta, reg, Y, Y_test):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    y_mean = np.mean(np.array([tup[2] for tup in Y]))
    
    # Initialize U, V  
    U = np.random.random((M,K)) - 0.5
    V = np.random.random((K,N)) - 0.5
    
    best_test_error = float('inf')
    
    size = len(Y)
    delta = None
    indices = np.arange(size)    
    epoch = 0

    
    while epoch < 301:
        epoch += 1
        
        # Run an epoch of SGD
        before_E_in = get_err(U, V, Y, reg)
        np.random.shuffle(indices)
        
        prev_U = U.copy()
        prev_V = V.copy()
        
        for ind in indices:
            (i,j, Yij) = Y[ind]
            # Update U[i], V[j]
            U[i-1] = grad_U(U[i-1], Yij, V[:,j-1], y_mean, reg, eta)
            V[:,j-1] = grad_V(V[:,j-1], Yij, U[i-1], y_mean, reg, eta);
        # At end of epoch, print E_in
        E_in = get_err(U, V, Y, reg)
        E_out = get_msqe_err(U, V, Y_test)

        print("Epoch %s, E_in (regularized MSE): %s; E_out (MSQE): %s"%(epoch, E_in, E_out))
#         print("Epoch %s, E_in (regularized MSE): %s; MSQE: %s"%(epoch + 1, E_in, get_msqe_err(U, V, Y)))


        if best_test_error > E_out:
            best_test_error = E_out
        else:
            return (prev_U, prev_V, get_msqe_err(prev_U, prev_V, Y), get_msqe_err(prev_U, prev_V, Y_test), epoch - 1)

#     return (U, V, get_err(U, V, Y,y_mean))
    return -1

In [4]:
ratings = np.array(lst)

# train/test

In [ ]:
Y_train = np.loadtxt('data/train.txt').astype(int)
Y_test = np.loadtxt('data/test.txt').astype(int)

In [ ]:
num_movies = len(movie_dic)
num_users = 943
m = num_users
n = num_movies
k = 20

etas = [0.01, 0.012, 0.008, 0.014, 0.006, 0.016, 0.004, 0.018, 0.002, 0.02, 0.001]
regs = [0.1, 0.12, 0.08, 0.14, 0.06, 0.16, 0.04, 0.18, 0.02, 0.2, 0.01]

best_pair = (1, 0.001)
best_e_out = float('inf')

for eta in etas:
    for reg in regs:
        u,v, final_err, final_test_err, epoch = train_model(m, n, k, eta, reg, Y_train, Y_test)
        
        print(eta, reg, final_test_err)
        
        if best_e_out > final_test_err:
            best_pair = (eta, reg)
            best_e_out = final_test_err
            
            print("new record")

print("Best (eta, reg) pair: ", best_pair)


u,v, final_err, final_test_err, epoch = train_model(m, n, k, best_pair[0], best_pair[1], Y_train, Y_test)

print(final_test_err, epoch)

Epoch 1, E_in (regularized MSE): 1.2140114933666433; E_out (MSQE): 1.1310849234494575
Epoch 2, E_in (regularized MSE): 1.1094355951076116; E_out (MSQE): 1.0980247087053132
Epoch 3, E_in (regularized MSE): 0.9707259801137404; E_out (MSQE): 1.048357276194229
Epoch 4, E_in (regularized MSE): 0.8701418844859088; E_out (MSQE): 1.0127264008097776
Epoch 5, E_in (regularized MSE): 0.8069466870678143; E_out (MSQE): 0.9913227432761118
Epoch 6, E_in (regularized MSE): 0.7643778478772624; E_out (MSQE): 0.9784242305834556
Epoch 7, E_in (regularized MSE): 0.733512288783273; E_out (MSQE): 0.9679563820467755
Epoch 8, E_in (regularized MSE): 0.7097207105871309; E_out (MSQE): 0.9616052893571578
Epoch 9, E_in (regularized MSE): 0.6918912514118819; E_out (MSQE): 0.9563875804387645
Epoch 10, E_in (regularized MSE): 0.6752001764878972; E_out (MSQE): 0.9515931476509538
Epoch 11, E_in (regularized MSE): 0.6624475973223881; E_out (MSQE): 0.9488757948226886
Epoch 12, E_in (regularized MSE): 0.6494045354673712; 

Epoch 12, E_in (regularized MSE): 0.5350631214540241; E_out (MSQE): 0.9538051214891559
Epoch 13, E_in (regularized MSE): 0.5219157968401745; E_out (MSQE): 0.9524197111435081
Epoch 14, E_in (regularized MSE): 0.5111358507443157; E_out (MSQE): 0.9523901784659128
Epoch 15, E_in (regularized MSE): 0.5005236066398935; E_out (MSQE): 0.9520879429091389
Epoch 16, E_in (regularized MSE): 0.4915638002642263; E_out (MSQE): 0.9517823190249233
Epoch 17, E_in (regularized MSE): 0.4838480752808979; E_out (MSQE): 0.9523145961336944
0.01 0.06 0.9517823190249233
Epoch 1, E_in (regularized MSE): 1.225069296096769; E_out (MSQE): 1.128512102254199
Epoch 2, E_in (regularized MSE): 1.1712230251961548; E_out (MSQE): 1.1096492514571659
Epoch 3, E_in (regularized MSE): 1.0799623433130605; E_out (MSQE): 1.0734314883081155
Epoch 4, E_in (regularized MSE): 0.9836305976150184; E_out (MSQE): 1.03527893160931
Epoch 5, E_in (regularized MSE): 0.9186502753563992; E_out (MSQE): 1.0094607120210517
Epoch 6, E_in (regulari

Epoch 5, E_in (regularized MSE): 0.639320588798198; E_out (MSQE): 1.0118670744140126
Epoch 6, E_in (regularized MSE): 0.5885402203096783; E_out (MSQE): 1.0068730016975185
Epoch 7, E_in (regularized MSE): 0.5504101378759162; E_out (MSQE): 1.0067810717170913
Epoch 8, E_in (regularized MSE): 0.5203628633150075; E_out (MSQE): 1.0075819376219883
0.01 0.01 1.0067810717170913
Epoch 1, E_in (regularized MSE): 1.2047770594518736; E_out (MSQE): 1.132490103907271
Epoch 2, E_in (regularized MSE): 1.0836242929977336; E_out (MSQE): 1.0916681981191345
Epoch 3, E_in (regularized MSE): 0.925631971583209; E_out (MSQE): 1.0335487002360757
Epoch 4, E_in (regularized MSE): 0.8279764371104497; E_out (MSQE): 0.9974309278459249
Epoch 5, E_in (regularized MSE): 0.7717383734190175; E_out (MSQE): 0.9775058760646134
Epoch 6, E_in (regularized MSE): 0.734228212225585; E_out (MSQE): 0.9647368152764491
Epoch 7, E_in (regularized MSE): 0.7060347820228253; E_out (MSQE): 0.9561454880276515
Epoch 8, E_in (regularized MS

Epoch 4, E_in (regularized MSE): 0.9548792428550448; E_out (MSQE): 1.0230952789689027
Epoch 5, E_in (regularized MSE): 0.8975776983759157; E_out (MSQE): 1.0019698383552111
Epoch 6, E_in (regularized MSE): 0.8594304850010148; E_out (MSQE): 0.9866406638750702
Epoch 7, E_in (regularized MSE): 0.8342770861818898; E_out (MSQE): 0.977635261627072
Epoch 8, E_in (regularized MSE): 0.8178217600638973; E_out (MSQE): 0.97173036329147
Epoch 9, E_in (regularized MSE): 0.8030103879652725; E_out (MSQE): 0.9665358543338649
Epoch 10, E_in (regularized MSE): 0.7920653925251494; E_out (MSQE): 0.9624065296920912
Epoch 11, E_in (regularized MSE): 0.7834666115076379; E_out (MSQE): 0.959448321553581
Epoch 12, E_in (regularized MSE): 0.7778689291842641; E_out (MSQE): 0.9575273963741213
Epoch 13, E_in (regularized MSE): 0.7733616200556921; E_out (MSQE): 0.9555520897005707
Epoch 14, E_in (regularized MSE): 0.7680206778428131; E_out (MSQE): 0.9536049974544264
Epoch 15, E_in (regularized MSE): 0.7639085990166253;

Epoch 13, E_in (regularized MSE): 0.6749308135144905; E_out (MSQE): 0.950173012706684
Epoch 14, E_in (regularized MSE): 0.6629056093814997; E_out (MSQE): 0.9473363972810955
Epoch 15, E_in (regularized MSE): 0.653424822432285; E_out (MSQE): 0.9456991052044903
Epoch 16, E_in (regularized MSE): 0.6438343784768926; E_out (MSQE): 0.9437874638441742
Epoch 17, E_in (regularized MSE): 0.6354198772999913; E_out (MSQE): 0.9421239721054363
Epoch 18, E_in (regularized MSE): 0.6276266398126292; E_out (MSQE): 0.9406614252860123
Epoch 19, E_in (regularized MSE): 0.6195396043888471; E_out (MSQE): 0.9398695940110935
Epoch 20, E_in (regularized MSE): 0.6131598023877737; E_out (MSQE): 0.93897555266489
Epoch 21, E_in (regularized MSE): 0.6074530081109677; E_out (MSQE): 0.938100929492809
Epoch 22, E_in (regularized MSE): 0.6015662441027697; E_out (MSQE): 0.937513690083738
Epoch 23, E_in (regularized MSE): 0.595540723358096; E_out (MSQE): 0.9366475333736605
Epoch 24, E_in (regularized MSE): 0.59087172426182

...

Epoch 26, E_in (regularized MSE): 0.937946164573558; E_out (MSQE): 1.0663163721151778
Epoch 27, E_in (regularized MSE): 0.9218408889375608; E_out (MSQE): 1.0618994936821184
Epoch 28, E_in (regularized MSE): 0.9062553441708208; E_out (MSQE): 1.0576754001454305
Epoch 29, E_in (regularized MSE): 0.8912125488671091; E_out (MSQE): 1.053636866409655
Epoch 30, E_in (regularized MSE): 0.8766988122616688; E_out (MSQE): 1.0497542082723699
Epoch 31, E_in (regularized MSE): 0.8627250554283863; E_out (MSQE): 1.0460273814226808
Epoch 32, E_in (regularized MSE): 0.8492715128615089; E_out (MSQE): 1.0424548788537351
Epoch 33, E_in (regularized MSE): 0.8363876555911253; E_out (MSQE): 1.0390672095852467
Epoch 34, E_in (regularized MSE): 0.82404866804467; E_out (MSQE): 1.0358496906598995
Epoch 35, E_in (regularized MSE): 0.8122428247862219; E_out (MSQE): 1.0327786245108843
Epoch 36, E_in (regularized MSE): 0.8009842663773257; E_out (MSQE): 1.029859022765131
Epoch 37, E_in (regularized MSE): 0.790242327216

Epoch 1, E_in (regularized MSE): 1.3599734556151049; E_out (MSQE): 1.170725821878421
Epoch 2, E_in (regularized MSE): 1.3233682280997037; E_out (MSQE): 1.1596493731386899
Epoch 3, E_in (regularized MSE): 1.2979943942988195; E_out (MSQE): 1.1520129810999495
Epoch 4, E_in (regularized MSE): 1.279704169985195; E_out (MSQE): 1.1465219668334483
Epoch 5, E_in (regularized MSE): 1.2660867919454528; E_out (MSQE): 1.1424152660524949
Epoch 6, E_in (regularized MSE): 1.2556253290782338; E_out (MSQE): 1.1392593734309646
Epoch 7, E_in (regularized MSE): 1.2473718697014737; E_out (MSQE): 1.1367558474512645
Epoch 8, E_in (regularized MSE): 1.2406834986497053; E_out (MSQE): 1.1346762386224305
Epoch 9, E_in (regularized MSE): 1.2351042415716842; E_out (MSQE): 1.1329105858233528
Epoch 10, E_in (regularized MSE): 1.2303179418186112; E_out (MSQE): 1.1313664915771058
Epoch 11, E_in (regularized MSE): 1.2260955959949515; E_out (MSQE): 1.1299628392027452
Epoch 12, E_in (regularized MSE): 1.2222509114978661; 

Epoch 96, E_in (regularized MSE): 0.8470169264324928; E_out (MSQE): 0.9811958745305053
Epoch 97, E_in (regularized MSE): 0.8458786006884893; E_out (MSQE): 0.9807126430348928
Epoch 98, E_in (regularized MSE): 0.8448429691926287; E_out (MSQE): 0.9802676752958844
Epoch 99, E_in (regularized MSE): 0.8438140252857367; E_out (MSQE): 0.9798283150066617
Epoch 100, E_in (regularized MSE): 0.8428132841898376; E_out (MSQE): 0.9793925967054125
Epoch 101, E_in (regularized MSE): 0.841842656740376; E_out (MSQE): 0.9789673126310448
Epoch 102, E_in (regularized MSE): 0.840835304118467; E_out (MSQE): 0.9785512163694869
Epoch 103, E_in (regularized MSE): 0.8399703377686025; E_out (MSQE): 0.9781363183847238
Epoch 104, E_in (regularized MSE): 0.8390505831483243; E_out (MSQE): 0.9777283182596542
Epoch 105, E_in (regularized MSE): 0.8381436721366983; E_out (MSQE): 0.9773670256223357
Epoch 106, E_in (regularized MSE): 0.8372811614359984; E_out (MSQE): 0.9769923280760806
Epoch 107, E_in (regularized MSE): 0.8

Epoch 190, E_in (regularized MSE): 0.8013273056902623; E_out (MSQE): 0.9606039962153452
Epoch 191, E_in (regularized MSE): 0.801076416610429; E_out (MSQE): 0.9605042486662441
Epoch 192, E_in (regularized MSE): 0.800875867844803; E_out (MSQE): 0.9604042981508872
Epoch 193, E_in (regularized MSE): 0.800671216658674; E_out (MSQE): 0.9603412907556066
Epoch 194, E_in (regularized MSE): 0.8004125578613227; E_out (MSQE): 0.9602201744165135
Epoch 195, E_in (regularized MSE): 0.8002529759523643; E_out (MSQE): 0.960147818062989
Epoch 196, E_in (regularized MSE): 0.8000548250104176; E_out (MSQE): 0.9600662911298035
Epoch 197, E_in (regularized MSE): 0.7998535801692286; E_out (MSQE): 0.9599703003653179
Epoch 198, E_in (regularized MSE): 0.7996640405331515; E_out (MSQE): 0.9598987346049559
Epoch 199, E_in (regularized MSE): 0.7994450195582653; E_out (MSQE): 0.9598231127649902
Epoch 200, E_in (regularized MSE): 0.7992858729386288; E_out (MSQE): 0.9597264946244737
Epoch 201, E_in (regularized MSE): 0

Epoch 53, E_in (regularized MSE): 0.6332494311507045; E_out (MSQE): 1.0047774578767787
Epoch 54, E_in (regularized MSE): 0.6280551472341533; E_out (MSQE): 1.0041191403785736
Epoch 55, E_in (regularized MSE): 0.6230289708136688; E_out (MSQE): 1.0035152146754966
Epoch 56, E_in (regularized MSE): 0.6181564940795717; E_out (MSQE): 1.0029697860696276
Epoch 57, E_in (regularized MSE): 0.6134221152145428; E_out (MSQE): 1.0024804729296075
Epoch 58, E_in (regularized MSE): 0.6088253333442516; E_out (MSQE): 1.0020184786385098
Epoch 59, E_in (regularized MSE): 0.6043560779763313; E_out (MSQE): 1.0016283579585892
Epoch 60, E_in (regularized MSE): 0.6000127401204989; E_out (MSQE): 1.001275859107891
Epoch 61, E_in (regularized MSE): 0.5957932297216562; E_out (MSQE): 1.000960215665809
Epoch 62, E_in (regularized MSE): 0.5916741915337238; E_out (MSQE): 1.0006965315817797
Epoch 63, E_in (regularized MSE): 0.5876699011380415; E_out (MSQE): 1.000432983177306
Epoch 64, E_in (regularized MSE): 0.5837701412

In [ ]:
with open("matrices/normal_best_model_eta_{}_reg_{}_epoch_{}_testerr_{}.matrix".format(best_pair[0], best_pair[1], epoch, final_test_err), "wb+") as f:
    np.save(f, v)

# SVD visualization

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
svd.fit_transform(v)
print("done")

In [ ]:
from numpy.linalg import svd
a, sigma, b = np.linalg.svd(v)

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
np.diag(sigma).shape

In [ ]:
u.shape

In [ ]:
v.shape

In [ ]:
transformer = a[:, :2].T
print(transformer.shape)

In [ ]:
transformed_v = np.matmul(transformer, v).T
print(transformed_v.shape)

In [ ]:
transformed_u = np.matmul(transformer, u.T).T
print(transformed_u.shape)

# visualizations

In [ ]:
indices = np.random.randint(1682, size=10)

In [ ]:
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]

In [ ]:
plt.scatter(x,y)

In [ ]:
num_ratings_vs_movies = [0 for i in movie_dic.keys()]
for (i, j, yij) in lst:
    num_ratings_vs_movies[j] += 1
num_ratings_vs_movies = np.array(num_ratings_vs_movies)

In [ ]:
indices = num_ratings_vs_movies.argsort()[-10:][::-1]

In [ ]:
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
movie_rating_dic = {id: [] for id in movie_dic.keys()} # dic of movie id and ratings

for rating in lst:
    _, movie_id, r = rating
    movie_rating_dic[movie_id].append(r)

avg_ratings = {movie: sum(movie_rating_dic[movie]) / len(movie_rating_dic[movie]) \
               for movie in movie_rating_dic}

# get ten best movies that received highest average rating
top_10 = sorted(avg_ratings.items(), key=lambda tup: tup[1], reverse=True)[:10]
indices = [t[0] for t in top_10]

In [ ]:
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
fant_movies = movie_genre_dic['Fantasy']
indices = fant_movies[:10]
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
fant_movies = movie_genre_dic['Documentary']
indices = fant_movies[:10]
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
fant_movies = movie_genre_dic["Children's"]
indices = fant_movies[:10]
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)